# Исследование препроцессинга на распознавание типа и номера документов прототипа OCR. Детекция OpenVINO. Распознавание Tesseract 
- Автор: Кирилл Киселев
- Дата начала: 11.05.2022
- Описание исследования: в датасете содержится 203 изображения. Сперва необходимо определить количество отфильтрованных изображений, затем получить результаты распознавания. Сделать выводы.

## Загрузка библиотек

In [153]:
import os
import pandas as pd
import cv2
import numpy as np

## Глобальные переменные

In [154]:
!pwd

/home/kirillk/PycharmProjects/useful_notebooks/ocr


In [456]:
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_1.txt'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_2.txt'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_3.txt'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_4.txt'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_5.txt'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_6.txt'
# TXT_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results_iter_7.txt'
# TXT_PATH = '/home/kirillk/PycharmProjects/ocr_prototype_tesseract/ocr-prototype/assets/results/ocr_ovino_tesseract_iter_1.txt'
# TXT_PATH = '/home/kirillk/PycharmProjects/ocr_prototype_tesseract/ocr-prototype/assets/results/ocr_ovino_tesseract_iter_2.txt'
# TXT_PATH = '/home/kirillk/PycharmProjects/ocr_prototype_tesseract/ocr-prototype/assets/results/ocr_ovino_tesseract_iter_3.txt'
# TXT_PATH = '/home/kirillk/PycharmProjects/ocr_prototype_tesseract/ocr-prototype/assets/results/ocr_ovino_tesseract_iter_4.txt'
TXT_PATH = '/home/kirillk/PycharmProjects/ocr_prototype_tesseract/ocr-prototype/assets/results/ocr_ovino_tesseract_iter_9.txt'

SAVE_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/paddle_ocr_results'
DATA_PATH = '/home/kirillk/datasets/OCR/ocr_test_cases/test_cases/'

## Загрузка данных и подготовка датасета

In [457]:
filename_list = []
results_list = []

with open(TXT_PATH, "r") as file:
    for num, line in enumerate(file):
        if 'jpg' in line:
            line = line.split(']')[-1].rstrip()
            filename_list.append(line)
        elif 'FPS' in line:
            break
        else:
            results_list.append(line.rstrip())
            
# results_list = results_list[0:-1]

In [458]:
filename_list

['108.jpg',
 '134.jpg',
 '145.jpg',
 '020.jpg',
 '172.jpg',
 '030.jpg',
 '044.jpg',
 '072.jpg',
 '183.jpg',
 '088.jpg',
 '192.jpg',
 '191.jpg',
 '122.jpg',
 '029.jpg',
 '144.jpg',
 '109.jpg',
 '049.jpg',
 '037.jpg',
 '138.jpg',
 '200.jpg',
 '189.jpg',
 '114.jpg',
 '195.jpg',
 '155.jpg',
 '093.jpg',
 '010.jpg',
 '148.jpg',
 '133.jpg',
 '199.jpg',
 '124.jpg',
 '032.jpg',
 '152.jpg',
 '171.jpg',
 '177.jpg',
 '075.jpg',
 '038.jpg',
 '000.jpg',
 '173.jpg',
 '126.jpg',
 '035.jpg',
 '003.jpg',
 '062.jpg',
 '094.jpg',
 '184.jpg',
 '004.jpg',
 '083.jpg',
 '164.jpg',
 '025.jpg',
 '101.jpg',
 '028.jpg',
 '076.jpg',
 '146.jpg',
 '067.jpg',
 '052.jpg',
 '087.jpg',
 '018.jpg',
 '051.jpg',
 '147.jpg',
 '187.jpg',
 '047.jpg',
 '115.jpg',
 '140.jpg',
 '079.jpg',
 '061.jpg',
 '096.jpg',
 '007.jpg',
 '011.jpg',
 '023.jpg',
 '120.jpg',
 '084.jpg',
 '065.jpg',
 '098.jpg',
 '130.jpg',
 '175.jpg',
 '022.jpg',
 '009.jpg',
 '069.jpg',
 '014.jpg',
 '054.jpg',
 '019.jpg',
 '056.jpg',
 '008.jpg',
 '017.jpg',
 '16

In [459]:
len(filename_list)

203

In [460]:
results_list

["{'id': 0, 'barcode': '2860587076540', 'doc_type': 'Товарная накладная', 'doc_number': '6017110800', 'error_list': ['Пересканируйте, текст не найден'], 'barcode_region': [161, 0, 1593, 5848], 'type_region': [1092, 3069, 1837, 3437], 'number_region': [1700, 3081, 2155, 3414]}",
 '{\'id\': 1, \'barcode\': \'2860592637279\', \'doc_type\': "Товарная| \'накладная", \'doc_number\': \'8017263078\', \'error_list\': [\'Пересканируйте, текст не найден\'], \'barcode_region\': [164, 0, 1622, 5900], \'type_region\': [1235, 756, 1994, 1127], \'number_region\': [1855, 767, 2318, 1104]}',
 "{'id': 2, 'barcode': '2860601366206', 'doc_type': 'Товарная| |накладная', 'doc_number': '—_—..KoluyecraoAee—', 'error_list': ['Пересканируйте, текст не найден'], 'barcode_region': [164, 0, 1628, 5913], 'type_region': [1181, 754, 1942, 1127], 'number_region': [1803, 766, 2268, 1103]}",
 "{'id': 3, 'barcode': '2810094798084', 'doc_type': 'Товарная накладная', 'doc_number': '6010358825', 'error_list': ['Пересканируйт

In [461]:
len(results_list)

203

In [462]:
df = pd.DataFrame()

for num, _ in enumerate(results_list):
    q = eval(results_list[num].replace('\\', ''))
    if q['error_list'] == []:
        q['error_list'] = 0
    elif len(q['error_list']) == 2:
        q['error_list'] = 2
    else:
        q['error_list'] = 1
        
    q['barcode_region'] = 0
    q['type_region'] = 0
    q['number_region'] =0
    
    res_df = pd.DataFrame(q, index=[0])
    df = pd.concat([df, res_df])

In [463]:
df = df.reset_index(drop=True)

In [464]:
df['filename'] = filename_list

In [465]:
df = df[['id', 'filename', 'barcode', 'doc_type', 'doc_number', 'error_list']]

In [466]:
df.head()

,id,filename,barcode,doc_type,doc_number,error_list
0,0,108.jpg,2860587076540,Товарная накладная,6017110800,1
1,1,134.jpg,2860592637279,Товарная| 'накладная,8017263078,1
2,2,145.jpg,2860601366206,Товарная| |накладная,—_—..KoluyecraoAee—,1
3,3,020.jpg,2810094798084,Товарная накладная,6010358825,1
4,4,172.jpg,2860614983032,Товарная| накладная,6017560483,1


## Визуализация

## Анализ данных

### Необходимые функции

In [467]:
def not_numeric_in_colunm(col):
    not_numeric = {}
    for i, value in enumerate(col):
        if col[i].isnumeric() == False:
            not_numeric[i] = value
    return not_numeric

In [468]:
def len_of_numbers(col):
    length_dict = {}
    for i, value in enumerate(col):
        if value.isnumeric():
            if len(value) not in length_dict.keys():
                length_dict[len(value)] = 1
            else:
                length_dict[len(value)] += 1
    return length_dict

### Error list = 2

Если в столбце **error_list** 0, значит изображение прошло проверку детектором блюра и предварительным распознавателем текста (определение ортогонального положения). 

Если в этом столбце 1, значит имеет место одна или две ошибки. следовательно изображение не будет распознано. Необходимо оценить качество работы детектора блюра и ортогонального оперделителя. 

In [469]:
df['error_list'].value_counts()

1    193
2     10
Name: error_list, dtype: int64

Итого не будет распознано **10** изображений. Оценим их.

In [470]:
error_df = df[df['error_list'] == 2]
error_df

,id,filename,barcode,doc_type,doc_number,error_list
22,22,195.jpg,2860615802998,Товарная` |накладная,None,2
23,23,155.jpg,2860609887666,Товарная| накладная,None,2
60,60,115.jpg,2860611107011,None,None,2
83,83,168.jpg,2860611147581,None,None,2
90,90,112.jpg,2860609826856,None,None,2
100,100,159.jpg,2860610411164,None,None,2
123,123,166.jpg,2860611250205,None,None,2
166,166,170.jpg,2860611500225,None,None,2
178,178,157.jpg,2860610590685,None,None,2
185,185,174.jpg,2860614951178,Товарная ‚накладная,$017561013,2


**Вывод**

Можно сказать, что деткетор блюра позволил отсеять размытые изображения, их загрузка в SAP нежелательная, поскольку большая часть информации в них потеряна.

### Error list = 1

In [471]:
good_df = df[df['error_list'] == 1]
good_df.head(10)

,id,filename,barcode,doc_type,doc_number,error_list
0,0,108.jpg,2860587076540,Товарная накладная,6017110800,1
1,1,134.jpg,2860592637279,Товарная| 'накладная,8017263078,1
2,2,145.jpg,2860601366206,Товарная| |накладная,—_—..KoluyecraoAee—,1
3,3,020.jpg,2810094798084,Товарная накладная,6010358825,1
4,4,172.jpg,2860614983032,Товарная| накладная,6017560483,1
5,5,030.jpg,2810095110823,Товарная ‘накладная,6010496453,1
6,6,044.jpg,2860350653671,Товарная накладная,6010495761,1
7,7,072.jpg,2860356028688,Товарная накладная,6010580640,1
8,8,183.jpg,2860618787148,Товарная ‚накладная,60176510871,1
9,9,088.jpg,2810078644574,Товарная накладная,6008222134,1


In [472]:
good_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 0 to 202
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          193 non-null    int64 
 1   filename    193 non-null    object
 2   barcode     193 non-null    object
 3   doc_type    192 non-null    object
 4   doc_number  192 non-null    object
 5   error_list  193 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 10.6+ KB


In [473]:
good_df  = good_df.fillna('None')

In [474]:
good_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193 entries, 0 to 202
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          193 non-null    int64 
 1   filename    193 non-null    object
 2   barcode     193 non-null    object
 3   doc_type    193 non-null    object
 4   doc_number  193 non-null    object
 5   error_list  193 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 10.6+ KB


In [475]:
good_df = good_df.reset_index(drop=True)

**Вывод**

Далее будем работать с этим датасетом. Отфильтрованные изображением считаем истинно-положительными.

### Barcode

In [476]:
bad_barcode_df = good_df[good_df['barcode'] == 'None'].reset_index(drop=True)
bad_barcode_df

,id,filename,barcode,doc_type,doc_number,error_list


In [477]:
barcode_not_numeric = not_numeric_in_colunm(good_df['barcode'])
barcode_not_numeric

{}

In [478]:
length_of_barcode_numbers = len_of_numbers(good_df['barcode'])
length_of_barcode_numbers

{13: 193}

In [479]:
length_of_barcode_numbers[13] / len(good_df)

1.0

In [480]:
bad_barcode_list = list(bad_barcode_df['filename'])
bad_barcode_list

[]

#### Вывод 
В столбце со штрих-кодами 0 пропущенных значений **None**. Остальные значения соответствуют десятизначным штрих-кодам. Итого можно считать распознанными **100%** штрих-кодов. Из исходного датасета необходимо исключить 10 изображений, которые содержали больше чем одну ошибку в списке error_list.

### Doc type

In [481]:
good_df['doc_type'].unique()

array(['Товарная накладная', "Товарная| 'накладная",
       'Товарная| |накладная', 'Товарная| накладная',
       'Товарная ‘накладная', 'Товарная ‚накладная',
       "Товарная' `накладная", 'Товарная |накладная',
       'Товарная` |Накладная', "Товарная: 'накладная",
       "Товарная' |накладная", 'Товарная` ‚накладная',
       'Товарная| ‚накладная', 'Товарная: `накладная', 'None',
       'Товарная. ‚накладная', 'Товарная‹ |накладная',
       'Товарная| [накладная', 'Товарная, |Ннакладная',
       'Товарная| `накладная', 'Товарная: накладная',
       "Товарная 'накладная", 'Товарная: |накладная',
       'Товарная, `накладная', 'Товарная` |накладная',
       'Товарная: гнакладная', 'Товарная. |накладная',
       'Товарная. накладная', 'Товарнал :накладная',
       'Товарная! |накладная', 'Товарная, ‚накладная',
       'Товарная| гнакладная', 'Товарная` ›накладная',
       'Товарная| внакладная', 'Товарная нНакладная',
       'Товарная, |накладная', "Товарная. 'накладная",
       'Това

In [482]:
good_df['doc_type'].value_counts()

Товарная накладная       97
Товарная| |накладная     14
Товарная |накладная      10
Товарная| `накладная      5
Товарная| накладная       5
Товарная: |накладная      5
Товарная: накладная       5
Товарная ‚накладная       4
Товарная: 'накладная      4
Товарная| 'накладная      4
Товарная` |накладная      3
Товарная| ‚накладная      3
Товарная: гнакладная      2
Товарная 'накладная       2
Товарная ‘накладная       2
None                      1
Товарная! |накладная      1
Товарная: |вакладная      1
Товарная/ накладная       1
Товарная` `накладная      1
Товарная! гнакладная      1
Товарная| |Ннакладная     1
Товарная. 'накладная      1
Товарная, |накладная      1
Товарная нНакладная       1
Товарная| внакладная      1
Товарная` ›накладная      1
Товарная| гнакладная      1
Товарная, ‚накладная      1
Товарнал :накладная       1
Товарная. ‚накладная      1
Товарная. накладная       1
Товарная. |накладная      1
Товарная' `накладная      1
Товарная, `накладная      1
Товарная` |Накладная

In [483]:
1 - len(good_df[good_df['doc_type'] == 'None']) / len(good_df)

0.9948186528497409

In [484]:
bad_doc_type_df = good_df[good_df['doc_type'] == 'None']
bad_doc_type_df

,id,filename,barcode,doc_type,doc_number,error_list
31,33,177.jpg,2860617614391,None,5017644991,1


In [485]:
bad_doc_type_list = list(bad_doc_type_df['filename'])
sorted(bad_doc_type_list)

['177.jpg']

1. iter_3: seventh_preprocess ['128.jpg', '141.jpg', '154.jpg', '162.jpg', '164.jpg', '171.jpg', '177.jpg', '178.jpg', '192.jpg']
2. iter_4: first+seventh ['177.jpg']

#### Вывод 
В столбце с типами документов 1 пропущенных значения **None**. Также в этом столбце есть почти распознанные данные, например **Товарная, ,накладная**, **Товарная: 'накладная** и другие. В почти распознанных данных допущено по 1-3 ошибки, так что, можно считать их проходящими по порогу распознавания. Итого качественно распознано 192 типов документов из 193 примеров, или всего **99.5%**.

### Doc number

In [486]:
bad_doc_number = good_df[good_df['doc_number'] == 'None']
bad_doc_number.head(15)

,id,filename,barcode,doc_type,doc_number,error_list
106,112,113.jpg,2860597420210,Товарная! |накладная,None,1


In [487]:
bad_doc_number_df = good_df[good_df['doc_number'].str.isnumeric() == False]
display(bad_doc_number_df.tail(20))
print(len(bad_doc_number_df))

,id,filename,barcode,doc_type,doc_number,error_list
43,45,083.jpg,2810096516426,Товарная ‚накладная,)AOKYMeHTa,1
48,50,076.jpg,2810096516365,Товарная накладная,)AOKYMeHTa,1
49,51,146.jpg,2860601473409,Товарная‹ |накладная,PAOKyMEHTR,1
59,62,079.jpg,2810097409826,Товарная ‚накладная,)JOKYMEHTa,1
67,70,065.jpg,2810097409826,Товарная: накладная,AIOKYMeHTa,1
73,76,069.jpg,2810096516365,Товарная накладная,601058914«,1
105,111,165.jpg,2860611233796,Товарнал :накладная,R—P—[-ROKYMEHTa:,1
106,112,113.jpg,2860597420210,Товарная! |накладная,None,1
107,113,180.jpg,2860618473492,Товарная накладная,DAOKYMEHT2,1
112,118,141.jpg,2860599552148,"Товарная, ‚накладная",3AGKYM@eHTa,1


24


In [488]:
len_not_match = []

for num, value in enumerate(good_df['doc_number']):
    if len(value) != 10:
        len_not_match.append(good_df['filename'][num])

In [489]:
len_not_match

['145.jpg',
 '183.jpg',
 '189.jpg',
 '184.jpg',
 '165.jpg',
 '113.jpg',
 '129.jpg',
 '141.jpg',
 '128.jpg',
 '116.jpg',
 '167.jpg',
 '123.jpg',
 '162.jpg',
 '136.jpg',
 '169.jpg']

In [490]:
len(len_not_match)

15

In [491]:
good_doc_number_df = good_df[good_df['doc_number'].str.isnumeric() == True]

In [492]:
bad_doc_number_filenames = set(list(bad_doc_number_df['filename']) + len_not_match)

In [493]:
len(bad_doc_number_filenames)

27

In [494]:
len(good_df) - len(bad_doc_number_filenames)

166

In [495]:
1 - len(bad_doc_number_filenames)/len(good_df)

0.8601036269430051

In [496]:
bad_doc_number_filenames

{'065.jpg',
 '069.jpg',
 '076.jpg',
 '079.jpg',
 '081.jpg',
 '083.jpg',
 '109.jpg',
 '113.jpg',
 '116.jpg',
 '119.jpg',
 '123.jpg',
 '128.jpg',
 '129.jpg',
 '136.jpg',
 '141.jpg',
 '145.jpg',
 '146.jpg',
 '148.jpg',
 '162.jpg',
 '165.jpg',
 '167.jpg',
 '169.jpg',
 '180.jpg',
 '183.jpg',
 '184.jpg',
 '189.jpg',
 '190.jpg'}

In [497]:
bad_doc_number_it_4 = ['008.jpg', '019.jpg', '038.jpg', '039.jpg', '041.jpg', '044.jpg', '047.jpg',
                       '052.jpg', '057.jpg', '065.jpg', '066.jpg', '068.jpg', '069.jpg', '074.jpg',
                       '076.jpg', '079.jpg', '080.jpg', '081.jpg', '082.jpg', '083.jpg', '084.jpg',
                       '091.jpg', '095.jpg', '099.jpg', '103.jpg', '105.jpg', '109.jpg', '113.jpg',
                       '116.jpg', '123.jpg', '125.jpg', '127.jpg', '128.jpg', '129.jpg', '130.jpg',
                       '131.jpg', '132.jpg', '136.jpg', '138.jpg', '141.jpg', '144.jpg', '145.jpg',
                       '146.jpg', '148.jpg', '150.jpg', '151.jpg', '158.jpg', '162.jpg', '164.jpg', 
                       '165.jpg', '167.jpg', '175.jpg', '176.jpg', '179.jpg', '180.jpg', '182.jpg',
                       '183.jpg', '184.jpg', '188.jpg', '189.jpg', '190.jpg', '191.jpg', '193.jpg', 
                       '194.jpg', '200.jpg', '201.jpg', '202.jpg']

In [498]:
bad_doc_number_it_5 = ['065.jpg', '069.jpg', '076.jpg', '079.jpg', '081.jpg', '083.jpg', '109.jpg', '113.jpg',
                       '116.jpg', '119.jpg', '123.jpg', '128.jpg', '129.jpg', '136.jpg', '141.jpg', '145.jpg',
                       '146.jpg', '148.jpg', '162.jpg', '165.jpg', '167.jpg', '169.jpg', '180.jpg', '183.jpg',
                       '184.jpg', '189.jpg', '190.jpg']

In [499]:
good_doc_numbers = good_df[~good_df['filename'].isin(bad_doc_number_it_5)]['doc_number']
good_doc_numbers

0      6017110800
1      8017263078
3      6010358825
4      6017560483
5      6010496453
          ...    
188    6010503817
189    6010584004
190    5017648128
191    6008204454
192    6600061199
Name: doc_number, Length: 166, dtype: object

In [500]:
good_doc_numbers.str.startswith('6').value_counts()

True     157
False      9
Name: doc_number, dtype: int64

1. iter_5: first+seventh+(убрал outputbase digits) - 27 нераспознанных изображений

#### Вывод 
В столбце с номерами документов 27 пропущенных значения. В них фигурируют либо **None** либо нечисловые. Качественно распознано 166 номеров документов из 193 примеров, или всего 86%.

## Результат исследования

In [501]:
bad_lists_union = sorted(list(set().union(bad_doc_type_list, bad_doc_number_filenames)))

In [502]:
len(bad_lists_union)

28

In [503]:
len(good_df) - len(bad_lists_union)

165

In [504]:
1 -len(bad_lists_union) / len(good_df)

0.8549222797927462

## Общий вывод

**Итерация 1** - отсутствует предобработка (первый препроцессинг). Можно считать качественно распознанными 179 документов из 193, что составляет **92.75%** из тестовых данных.

**Итерация 2** - добавлен второй препроцессинг. Можно считать качественно распознанными 178 документов из 193, что составляет **92.70%** из тестовых данных.

**Итерация 3** - добавлен третий препроцессинг. Можно считать качественно распознанными 179 документов из 193, что составляет **92.75%** из тестовых данных.

**Итерация 4** - добавлен четвертый препроцессинг. Можно считать качественно распознанными 180 документов из 193, что составляет **93.26%** из тестовых данных.

**Итерация 5** - добавлен пятый препроцессинг. Можно считать качественно распознанными 180 документов из 193, что составляет **93.26%** из тестовых данных.

**Итерация 6** - добавлен новый препроцессинг. 

`return cv2.GaussianBlur(img, (1, 1), 0)`

Можно считать качественно распознанными 179 документов из 193, что составляет **92.75%** из тестовых данных.

In [150]:
bad_lists_union

['038.jpg',
 '039.jpg',
 '041.jpg',
 '044.jpg',
 '047.jpg',
 '057.jpg',
 '079.jpg',
 '082.jpg',
 '091.jpg',
 '095.jpg',
 '096.jpg',
 '099.jpg',
 '109.jpg',
 '113.jpg',
 '116.jpg',
 '119.jpg',
 '125.jpg',
 '127.jpg',
 '131.jpg',
 '132.jpg',
 '136.jpg',
 '138.jpg',
 '141.jpg',
 '144.jpg',
 '145.jpg',
 '146.jpg',
 '148.jpg',
 '150.jpg',
 '158.jpg',
 '164.jpg',
 '165.jpg',
 '167.jpg',
 '169.jpg',
 '175.jpg',
 '177.jpg',
 '179.jpg',
 '180.jpg',
 '182.jpg',
 '184.jpg',
 '188.jpg',
 '190.jpg',
 '193.jpg',
 '201.jpg',
 '202.jpg']

In [445]:
bad_doc_number_it_5 = ['065.jpg', '069.jpg', '076.jpg', '079.jpg', '081.jpg', '083.jpg', '109.jpg', '113.jpg',
                       '116.jpg', '119.jpg', '123.jpg', '128.jpg', '129.jpg', '136.jpg', '141.jpg', '145.jpg',
                       '146.jpg', '148.jpg', '162.jpg', '165.jpg', '167.jpg', '169.jpg', '180.jpg', '183.jpg',
                       '184.jpg', '189.jpg', '190.jpg']

In [448]:
bad_doc_number_it_5 = ['184.jpg', '189.jpg', '190.jpg']

In [452]:
for i in bad_doc_number_it_5:
    image = cv2.imread(DATA_PATH + i)
    kernel = np.ones((1, 1), 'uint8')
    
#     image = cv2.GaussianBlur(image, (3, 3), 1)
    image = cv2.dilate(image, kernel, iterations=2)
#     image = cv2.GaussianBlur(image, (5, 5), 4)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image = cv2.threshold(image, 140, 255, cv2.THRESH_BINARY)  # 130
    image = cv2.resize(image, (0, 0), fx=0.5, fy=0.5)
    cv2.imshow(f'Image {i}', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [455]:
for i in bad_doc_number_it_5:
    image = cv2.imread(DATA_PATH + i)
    kernel = np.ones((1, 1), 'uint8')
    
    image = cv2.GaussianBlur(image, (7, 7), 1)
#     image = cv2.dilate(image, kernel, iterations=2)
#     image = cv2.GaussianBlur(image, (5, 5), 4)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     _, image = cv2.threshold(image, 140, 255, cv2.THRESH_BINARY)  # 130
    image = cv2.resize(image, (0, 0), fx=0.5, fy=0.5)
    cv2.imshow(f'Image {i}', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Notes